In [1]:
%run stdPackages.ipynb

# Download data from ENTSOE

In [2]:
from entsoe.exceptions import NoMatchingDataError, InvalidBusinessParameterError, InvalidPSRTypeError
client = EntsoePandasClient(api_key=entsoeToken, retry_count = 5, retry_delay = 5)

## 1. Settings

Notes:
* Currently, we can only use 1 hour frequencies.
* We can atmost query 1 year worth of data (per query).

Specify time horizon for download (```YYYYMMDD```):

In [3]:
settings = {}
settings['tz'] = 'Europe/Brussels'
settings['tStart'] = {'year': '2023', 'month': '01', 'day': '01'}
settings['tEnd'] = {'year': '2023', 'month': '01', 'day': '31'}
start = pd.Timestamp(''.join(settings['tStart'].values()), tz='Europe/Brussels')
end = pd.Timestamp(''.join(settings['tEnd'].values()), tz='Europe/Brussels')

Include bidding areas:

In [4]:
g = entsoe.mappings.Area._member_names_.copy()

*Load data from previous run:*

In [5]:
# files = ['dayAheadPrices', 'netPositions', 'crossBorderFlows', 'scheduledExchanges', 'NTC_DA', 'NTC_WA', 'NTC_MA', 'NTC_YA', 'NTC_ID']
# filesDict = {}
# for i in range(len(files)):
#     with open(os.path.join(d['data'], f'FILE_{i}'), "rb") as file:
#         filesDict[files[i]] = pickle.load(file)

## 2. Get all data:

Store in dict. Store potential errors in separate dict to check later.

In [5]:
f = {}

## 2.1. Bidding-area data:

*Day ahead prices: Unit €/MWh. Frequency unit 1 hour.* 

In [6]:
f['PriceDayAhead'] = dict.fromkeys(g)
iterThrough = [k for k,v in f['PriceDayAhead'].items() if v is None]
for k in iterThrough:
    try:
        f['PriceDayAhead'][k] = client.query_day_ahead_prices(k, start = start, end = end)
    except NoMatchingDataError:
        f['PriceDayAhead'].pop(k)

Connection Error, retrying in 5 seconds


*Net position: MW imbalance in day ahead market for area. Negative value indicates that an area requires imports to balance the system. Frequency 1 hour.*

In [ ]:
f['NetPosition'] = dict.fromkeys(g)
iterThrough = [k for k,v in f['NetPosition'].items() if v is None]
for k in iterThrough:
    try:
        f['NetPosition'][k] = client.query_net_position(k, start = start, end = end, dayahead = True)
    except NoMatchingDataError:
        f['NetPosition'].pop(k)

## 2.2. Transfer capacities/trade stuff

**NOTE:** The different data on crossborder flows and net transfer capacities are not defined over the same set of areas. For instance:
* Crossborder flows and scheduledExchanges are generally defined for the same tuples (g, g').
* Net transfer capacities dayahead are defined over much fewer combinations (at least for 2024 data). This is also the case online - it is not an issue with the API calls. 
* Generally, the net transfer capacities for day-ahead, week-ahead, month-ahead, and year-head are all defined over different combinations. Once again, this is also the case for the data online.

All combinations from "neigbours" mapping:

In [ ]:
g2gg = [(k,v) for k in entsoe.mappings.NEIGHBOURS for v in entsoe.mappings.NEIGHBOURS[k]]

*Crossborder flows: MWh. Frequency unit 1 hour.* 

In [ ]:
f['CrossBorderFlows'] = dict.fromkeys(g2gg)
for k in g2gg:
    try:
        f['CrossBorderFlows'][(k[0], k[1])] = client.query_crossborder_flows(k[0], k[1], start=start, end=end)
    except (NoMatchingDataError, InvalidBusinessParameterError, InvalidPSRTypeError) as e:
        f['CrossBorderFlows'].pop(k)

*Crossborder flows, expected: MWh. Frequency unit 1 hour.* 

In [ ]:
f['ScheduleEx'] = dict.fromkeys(g2gg)
for k in g2gg:
    try:
        f['ScheduleEx'][(k[0], k[1])] = client.query_crossborder_flows(k[0], k[1], start=start, end=end, dayahead = False)
    except (NoMatchingDataError, InvalidBusinessParameterError, InvalidPSRTypeError) as e:
        f['ScheduleEx'].pop(k)

*Net transfer capacity, day ahead:*

In [ ]:
f['NTC_DA'] = dict.fromkeys(g2gg)
for k in g2gg:
    try:
        f['NTC_DA'][(k[0], k[1])] = client.query_net_transfer_capacity_dayahead(k[0], k[1], start=start, end=end)
    except (NoMatchingDataError, InvalidBusinessParameterError, InvalidPSRTypeError) as e:
        f['NTC_DA'].pop(k)

*Net transfer capacity, week ahead:*

In [ ]:
f['NTC_WA'] = dict.fromkeys(g2gg)
for k in g2gg:
    try:
        f['NTC_WA'][(k[0], k[1])] = client.query_net_transfer_capacity_weekahead(k[0], k[1], start=start, end=end)
    except (NoMatchingDataError, InvalidBusinessParameterError, InvalidPSRTypeError) as e:
        f['NTC_WA'].pop(k)

*Net transfer capacity, month ahead:*

In [ ]:
f['NTC_MA'] = dict.fromkeys(g2gg)
for k in g2gg:
    try:
        f['NTC_MA'][(k[0], k[1])] = client.query_net_transfer_capacity_monthahead(k[0], k[1], start=start, end=end)
    except (NoMatchingDataError, InvalidBusinessParameterError, InvalidPSRTypeError) as e:
        f['NTC_MA'].pop(k)

*Net transfer capacity, year ahead:*

In [ ]:
f['NTC_YA'] = dict.fromkeys(g2gg)
for k in g2gg:
    try:
        f['NTC_YA'][(k[0], k[1])] = client.query_net_transfer_capacity_yearahead(k[0], k[1], start=start, end=end)
    except (NoMatchingDataError, InvalidBusinessParameterError, InvalidPSRTypeError) as e:
        f['NTC_YA'].pop(k)

*Offered capacity, intraday:*

In [ ]:
storeExceptions = {}

In [ ]:
f['NTC_ID'] = dict.fromkeys(g2gg)
storeExceptions['NTC_ID'] = {}
for k in g2gg:
    try:
        f['NTC_ID'][(k[0], k[1])] = client.query_intraday_offered_capacity(k[0], k[1], start=start, end=end, implicit=True)
    except (NoMatchingDataError, InvalidBusinessParameterError, InvalidPSRTypeError) as e:
        f['NTC_ID'].pop(k)
    except Exception as e:
        storeExceptions['NTC_ID'][k] = e
        f['NTC_ID'].pop(k)

*Offered capacity:*

In [ ]:
f['NTC'] = dict.fromkeys(g2gg)
storeExceptions['NTC'] = {}
for k in g2gg:
    try:
        f['NTC'][(k[0], k[1])] = client.query_offered_capacity(k[0], k[1],"A01", start=start, end=end, implicit=True)
    except (NoMatchingDataError, InvalidBusinessParameterError, InvalidPSRTypeError) as e:
        f['NTC'].pop(k)
    except Exception as e:
        storeExceptions['NTC'][k] = e
        f['NTC'].pop(k)

*Water reservoirs for hydro storage (agg):*

In [ ]:
f['WaterRes'] = dict.fromkeys(g)
storeExceptions['WaterRes'] = {}
for k in g:
    try:
        f['WaterRes'][k] = client.query_aggregate_water_reservoirs_and_hydro_storage(k, start = start, end = end)
    except (NoMatchingDataError, InvalidBusinessParameterError, InvalidPSRTypeError) as e:
        f['WaterRes'].pop(k)
    except Exception as e:
        storeExceptions['WaterRes'][k] = e
        f['WaterRes'].pop(k)

Store:

In [ ]:
with open(os.path.join(d['data'], 'test'), "wb") as file:
    pickle.dump(f, file)